In [1]:
from router import router_api
import dspy
from langchain_openai import ChatOpenAI
router_api('mining')

lm = dspy.LM('openai/gpt-4.1-mini')
dspy.configure(lm=lm)
chat_model = ChatOpenAI(model='gpt-4.1-mini')
database_name = 'labeled'
target_dir = 'articles_processed'

/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
from ast import literal_eval
from typing import Optional, Literal
from functools import partial

from pydantic import BaseModel, Field, field_validator

from sisyphus.heas.label import label_paras
from sisyphus.chain.paragraph import Paragraph, ParagraphExtend
from sisyphus.chain import Filter, Writer
from sisyphus.strategy.run_strategy import ExtractStrategy
from sisyphus.strategy.pydantic_models_general import Processing, Material, MaterialDescriptionBase
from sisyphus.strategy.utils import get_paras_with_props, get_synthesis_paras
from sisyphus.heas.prompt import *
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.heas.synthesis import get_synthesis_prompt
from sisyphus.strategy.llm_models import categorize_agent


In [3]:
from typing import Literal
from sisyphus.utils.helper_functions import get_title_abs, render_docs

class ClassifyPaper(dspy.Signature):
    """assign label to HEAs (high entropy alloys) paper based on their title and abstract."""
    context: str = dspy.InputField(desc='Title and abstract of the paper')
    label: Literal['hea_experimental', 'hea_theoretical', 'irrelevant'] = dspy.OutputField(desc="Pay attention to keywords such as 'molecular dynamics' or 'machine learning,' which should be labeled as hea_theoretical. Label keywords related to fabrication processes as hea_experimental.")
    mechanical_relevancy: bool = dspy.OutputField(desc='whether this paper describe the mechanical properties such as tensile or compressive')
classifier_paper = dspy.ChainOfThought(signature=ClassifyPaper)

def paper_filter(docs):
    title, abstract = get_title_abs(docs)
    prediction = classifier_paper(context=render_docs(abstract, title))
    if prediction.label == 'hea_experimental' and prediction.mechanical_relevancy:
        return docs
    return

In [ ]:
# database = get_plain_articledb(database_name)
# getter = Filter(database)
# target = database_name + '_'
# result_db = get_plain_articledb(target)
# result_db.create_db()
# from sisyphus.heas.label import label_grain_size
# def load(docs):
#     return [Paragraph.from_labeled_document(doc, id_) for id_, doc in enumerate(docs)]
# def save(paras):
#     result_db.dump_state(paras)
# chain = getter + load + label_grain_size + save
# from sisyphus.chain.chain_elements import run_chains_with_extarction_history_multi_threads
# run_chains_with_extarction_history_multi_threads(chain, target_dir, 50, target)

100%|██████████| 1531/1531 [01:51<00:00, 13.76it/s]


In [ ]:
db = get_plain_articledb('heas_1531')
getter = Filter(db)
labeled_database = get_plain_articledb(database_name)
labeled_database.create_db()
def save(paras):
    labeled_database.dump_state(paras)
label_chain = getter + paper_filter + label_paras + save

In [ ]:
from sisyphus.chain.chain_elements import run_chains_with_extarction_history_multi_threads
run_chains_with_extarction_history_multi_threads(label_chain, target_dir, 5, database_name)